In [ ]:
import sqlite3
class Urun:
    def __init__(self, tablo_adi):
        self.tablo_adi = tablo_adi
        self.con = sqlite3.connect('restorant.db')
        self.cur = self.con.cursor()
        self.create_table()
    def create_table(self):
        self.cur.execute(f'''create table if not exists {self.tablo_adi}(
        id integer primary key autoincrement,
        isim  varchar(200) not null,
        fiyat real not null
        )    
        ''')
        self.con.commit()
    def urun_ekle(self, isim, fiyat):
        self.cur.execute(f" insert into {self.tablo_adi}(isim,fiyat) values(?,?)",(isim,fiyat))
        self.con.commit()
        print(f"{isim} urun eklendi.")
    def urun_sil(self, urun_id):
        self.cur.execute(f"delete from {self.tablo_adi} where id = ?",(urun_id,))
        self.con.commit()
        print(f"{urun_id} ilgili urun silindi.")
    def urun_guncelleme(self,urun_id,isim,fiyat):
        self.cur.execute(f" update {self.tablo_adi} set isim=?, fiyat=? where id=?",(isim,fiyat,urun_id))
        self.con.commit()
        print(f"{urun_id} ilgili urun guncellendi.")
    def urunleri_listele(self):
        self.cur.execute(f'select id,isim,fiyat from {self.tablo_adi}')
        urunler = self.cur.fetchall()
        print(f"##### {self.tablo_adi.upper()} #####")
        for urun in urunler:
            print(f"Id:{urun[0]}, isim{urun[1]}, fiyat:{urun[2]} TL")
class Yemek(Urun):
    def __init__(self):
        super().__init__("Yemekler")
class Icecek(Urun):
    def __init__(self):
        super().__init__("Icecekler")
class Siparisler:
    def __init__(self):
        self.con = sqlite3.connect('restorant.db')
        self.cur = self.con.cursor()
        self.create_table()
    def create_table(self):
        self.cur.execute(f'''create table if not exists Siparisler(
        id integer primary key autoincrement,
        yemek_id integer not null,
        yemek_adet integer not null,
        icecek_id integer not null,
        icecek_adet integer not null,
        toplam_fiyat real not null,
        foreign key (yemek_id) references yemekler(id),
        foreign key (icecek_id) references icecekler(id)
        )
        ''')
        self.con.commit()
    def siparis_ver(self, yemek_id, icecek_id, yemek_adet, icecek_adet):
        yemek_fiyat = self.get_urun_fiyat('yemekler',yemek_id)
        icecek_fiyat = self.get_urun_fiyat('icecekler',icecek_id)
        toplam_fiyat = (yemek_fiyat * yemek_adet) + (icecek_fiyat * icecek_adet)
        self.cur.execute(f"insert into siparisler (yemek_id, yemek_adet, icecek_id,icecek_adet,toplam_fiyat) values (?,?,?,?,?)",
                        (yemek_id,yemek_adet,icecek_id,icecek_adet,toplam_fiyat))
        self.con.commit()
        print(f""" 
        {yemek_adet} adet {self.get_urun_isimi('yemekler',yemek_id)} 
        {icecek_adet} adet {self.get_urun_isimi('icecekler',icecek_id)} siparisi alindi. 
        Toplam fiyat: {toplam_fiyat} TL.
        """)
    def siparisleri_goster(self):
        self.cur.execute('''select Siparisler.id, Yemekler.isim, icecekler.isim,
        Siparisler.yemek_adet, Siparisler.icecek_adet, Siparisler.toplam_fiyat
        from Siparisler 
        inner join Yemekler on Siparisler.yemek_id = Yemekler.id
        inner join Icecekler on Siparisler.icecek_id = Icecekler.id
        ''')
        siparisler = self.cur.fetchall()
        if not siparisler:
            print("Siparis bulunmamaktadir.")
        else:
            print(15 * "#", "Siparisler", 15 * "#")
            for siparis in siparisler:
                print(f""" 
                Id:{siparis[0]},
                Yemek:{siparis[1]},
                Yemek Adet:{sipari[3]},
                Icecek:{siparis[2]},
                Icecek Adet:{siparis[4]},
                Toplam Fiyat:{siparis[5]}
                """)
    def get_urun_fiyat(self,tablo,urun_id):
        self.cur.execute(f"select fiyat from {tablo} where id = ?",(urun_id,))
        fiyat = self.cur.fetchone()
        if fiyat:
            return fiyat[0]
        else:
            print("İlgili urun bulunamadi.")
            return 0
    def get_urun_isimi(self,tablo,urun_id):
        self.cur.execute(f"select isim from {tablo} where id = ?",(urun_id,))
        isim = self.cur.fetchone()
        if isim:
            return isim[0]
        else:
            print("İlgili urun bulunamadi.")
            return ""
class Restorant:
    def __init__(self):
        self.yemekler = Yemek()
        self.icecekler = Icecek()
        self.siparisler = Siparisler()
    def siparis_ver(self):
        self.yemekler.urunleri_listele()
        yemek_id = int(input("İstediginiz yemegin id'sini girin: "))
        yemek_adet = int(input("Siparis ettiginiz yemek adetini girin: "))
        self.icecekler.urunleri_listele()
        icecek_id = int(input("İstediginiz icecek id'sini girin: "))
        icecek_adet = int(input("Siparis ettiginiz icecek adetini girin: "))
        self.siparisler.siparis_ver(yemek_id,yemek_adet,icecek_id,icecek_adet)
    def menu_goster(self):
        while True:
            print(20 * "*", "Restorant Yonetim Sistemi", 20 * "*")
            print("""
            1. Yemek Ekle
            2. Icecek Ekle
            3. Yemek Sil
            4. Icecek Sil
            5. Yemek Guncelle
            6. Icecek Guncelle
            7. Yemekler
            8. Icecekler
            9. Siparisler
            10.Siparis ver
            0. Cikis
            """)
            islem = input("Yapmak istediginiz islem numarasini girin: ")
            if islem == "1":
                self.yemekler.urun_ekle(input("Yemek isimi:"), float(input("Yemek Fiyati: ")))
            elif islem == "2":
                self.icecekler.urun_ekle(input("İcecek isimi:"), float(input("İcecek Fiyati: ")))
            elif islem == "3":
                self.yemekler.urunleri_listele()
                self.yemekler.urun_sil(int(input("Silmek istediginiz yemegin id'sini girin: ")))
            elif islem == "4":
                self.icecekler.urunleri_listele()
                self.icecekler.urun_sil(int(input("Silmek istediginiz icecegin id'sini girin: ")))
            elif islem == "5":
                self.yemekler.urunleri_listele()
                urun_id = int(input("Guncellemek istedigini yemek id'sini girin: "))
                self.yemekler.urun_guncelleme(urun_id, input("Yeni yemek isimi: "), float(input("Yemegin yeni fiyati: ")))
            elif islem == "6":
                self.icecekler.urunleri_listele()
                urun_id = int(input("Guncellemek istedigin icecek id'sini girin: "))
                self.icecekler.urun_guncelleme(urun_id, input("Yeni icecek isimi: "), float(input("İcecegin yeni fiyati: ")))
            elif islem == "7":
                self.yemekler.urunleri_listele()
            elif islem == "8":
                self.icecekler.urunleri_listele()
            elif islem == "9":
                self.siparisler.siparisleri_goster()
            elif islem == "10":
                self.siparis_ver()
            elif islem == "0":
                print("Cikis yapildi.")
                break
            else:
                print("Gecersiz islem numarasi girildi.")
if __name__ == "__main__":
    restorant = Restorant()
    restorant.menu_goster()

******************** Restorant Yonetim Sistemi ********************

            1. Yemek Ekle
            2. Icecek Ekle
            3. Yemek Sil
            4. Icecek Sil
            5. Yemek Guncelle
            6. Icecek Guncelle
            7. Yemekler
            8. Icecekler
            9. Siparisler
            10.Siparis ver
            0. Cikis
            
Yapmak istediginiz islem numarasini girin: 10
##### YEMEKLER #####
Id:2, isimmercimek, fiyat:12.0 TL
İstediginiz yemegin id'sini girin: 2
Siparis ettiginiz yemek adetini girin: 1
##### ICECEKLER #####
Id:2, isimsalgam, fiyat:8.0 TL
İstediginiz icecek id'sini girin: 2
Siparis ettiginiz icecek adetini girin: 1
İlgili urun bulunamadi.
İlgili urun bulunamadi.
 
        2 adet mercimek 
        1 adet  siparisi alindi. 
        Toplam fiyat: 24.0 TL.
        
******************** Restorant Yonetim Sistemi ********************

            1. Yemek Ekle
            2. Icecek Ekle
            3. Yemek Sil
            4. Icecek Si